In [24]:
import os
import pandas as pd
import requests

# set working directory and pull api key
os.chdir("C:\\census_test")
with open('api_key.txt') as k:
    apikey=k.read().strip()

In [25]:
# url to retrieve the data groups list
grps_url = f'https://api.census.gov/data/2018/acs/acs1/subject/groups.html'
grps = pd.read_html(grps_url)
grps = pd.DataFrame(grps[0])
grps.head()

,Name,Description,Variable List,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,S0101,AGE AND SEX,selected variables,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,S0102,POPULATION 60 YEARS AND OVER IN THE UNITED STATES,selected variables,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,S0102PR,POPULATION 60 YEARS AND OVER IN PUERTO RICO,selected variables,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,S0103,POPULATION 65 YEARS AND OVER IN THE UNITED STATES,selected variables,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,S0103PR,POPULATION 65 YEARS AND OVER IN PUERTO RICO,selected variables,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Looking for poverty data - table name
grps[grps['Description'].str.contains("POVERTY")].reset_index()

,index,Name,Description,Variable List,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,34,S1701,POVERTY STATUS IN THE PAST 12 MONTHS,selected variables,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,35,S1702,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES,selected variables,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36,S1703,SELECTED CHARACTERISTICS OF PEOPLE AT SPECIFIE...,selected variables,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# function to retrieve group labels based on the selected table group
def grp_labels_df(selected_grp):
    grp_labels_url = f'https://api.census.gov/data/2018/acs/acs1/subject/groups/{selected_grp}.html'
    grp_labels = pd.read_html(grp_labels_url)
    grp_labels = pd.DataFrame(grp_labels[0])
    grp_labels['Label'].replace({"!!": " ", ":": ""}, regex=True, inplace=True)
    return grp_labels

In [28]:
# For poverty selected table name is S1701 - which has poverty status data in it
## retrieving its labels
grp_labels = grp_labels_df('S1701')

In [29]:
## Estimate of the number of people of any age in poverty - required variable
vars1 = grp_labels[grp_labels['Label'].str.contains("Estimate Total Population")].reset_index()
vars1 = vars1.loc[vars1.index[0],'Name']
vars1

'S1701_C01_001E'

In [30]:
## Estimate of the number of people under the age of 18 in poverty - required variable
vars2 = grp_labels[grp_labels['Label'].str.contains("AGE Under 18 years")].reset_index()
vars2 = vars2[vars2['Label'].str.contains("Estimate Total")].reset_index()
vars2 = vars2.loc[vars2.index[0],'Name']
vars2

'S1701_C01_002E'

In [31]:
##### Looking for Median income - table names
grps[grps['Description'].str.contains("INCOME")].reset_index()

,index,Name,Description,Variable List,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,39,S1901,INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATIO...,selected variables,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40,S1902,MEAN INCOME IN THE PAST 12 MONTHS (IN 2018 INF...,selected variables,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,41,S1903,MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2018 I...,selected variables,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
## selected table for median income data - S1901 - has median household income data
## retreiving its labels
selected_grp = 'S1901'
grp_labels_inc = grp_labels_df('S1901')

In [33]:
#Estimate of the median household income - required variable
vars3 = grp_labels_inc[grp_labels_inc['Label'].str.contains("Median income")].reset_index()
vars3 = vars3[vars3['Label'].str.contains("Estimate")].reset_index()
vars3 = vars3.loc[vars3.index[0],'Name']
vars3

'S1901_C01_012E'

In [34]:
####### Final Poverty data retrieval ##########
host_site = 'https://api.census.gov/data'
year = '/2018'
dataset = '/acs/acs1/subject'
get_fn = '?get='
variable_list = vars1+','+vars2
location = '&for=county:201&in=state:48' # harris county, TX state
api_key = f"&key={apikey}"

data_url = f"{host_site}{year}{dataset}{get_fn}{variable_list}{location}{api_key}"

In [37]:
pov_data = requests.get(data_url)
print(pov_data.content)

b'[["S1701_C01_001E","S1701_C01_002E","state","county"],\n["4650812","1238708","48","201"]]'


In [38]:
def json_df_fn(df):
    final_df = pd.DataFrame.from_dict(df.json())
    final_df.columns = final_df.iloc[0]
    final_df = final_df[1:].reset_index()
    final_df = final_df.iloc[: , 1:]
    return final_df

In [39]:
pov_data_df = json_df_fn(pov_data)

In [40]:
pov_data_df.rename(columns = {'S1701_C01_001E':'Poverty_All_Ages','S1701_C01_002E':'Poverty_Under_18Yrs'}, inplace = True)
pov_data_df

,Poverty_All_Ages,Poverty_Under_18Yrs,state,county
0,4650812,1238708,48,201


In [41]:
######### Final Median Income data retrieval ##########b
variable_list = vars3
data_url = f"{host_site}{year}{dataset}{get_fn}{variable_list}{location}{api_key}"
inc_data = requests.get(data_url)
print(inc_data.content)

b'[["S1901_C01_012E","state","county"],\n["60232","48","201"]]'


In [42]:
inc_data_df = json_df_fn(inc_data)

In [43]:
inc_data_df.rename(columns = {'S1901_C01_012E':'Median_household_income'}, inplace = True)
inc_data_df

,Median_household_income,state,county
0,60232,48,201


In [44]:
# Merge poverty and income data
final_df = pd.merge(pov_data_df, inc_data_df, how="inner", on=['state','county'])
final_df

,Poverty_All_Ages,Poverty_Under_18Yrs,state,county,Median_household_income
0,4650812,1238708,48,201,60232


In [45]:
final_df = final_df[['state','county','Poverty_All_Ages','Poverty_Under_18Yrs','Median_household_income']]

In [46]:
final_df.dtypes

0
state                      object
county                     object
Poverty_All_Ages           object
Poverty_Under_18Yrs        object
Median_household_income    object
dtype: object

In [47]:
final_df.state[final_df.state == '48'] = 'TX' 
final_df.county[final_df.county == '201'] = 'Harris County'
final_df['Year'] = 2018

C:\Users\Guggalla\AppData\Local\Temp\ipykernel_6196\2576106427.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.state[final_df.state == '48'] = 'TX'
C:\Users\Guggalla\AppData\Local\Temp\ipykernel_6196\2576106427.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.county[final_df.county == '201'] = 'Harris County'
C:\Users\Guggalla\AppData\Local\Temp\ipykernel_6196\2576106427.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [48]:
# write as csv
final_df.to_csv('output.csv', index=False)